In [ ]:
# !pip install beautifulsoup4

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [beautifulsoup4]

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [4]:
import requests
from bs4 import BeautifulSoup
import json
import time
import re
from urllib.parse import unquote
from tqdm import tqdm

class WikipediaCarParser:
    def __init__(self):
        self.base_url = "https://ru.wikipedia.org"
        self.api_url = "https://ru.wikipedia.org/w/api.php"
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        })
        self.articles = []
        
    def get_category_members(self, category, limit=500):
        """Получает список страниц из категории"""
        members = []
        params = {
            "action": "query",
            "list": "categorymembers",
            "cmtitle": category,
            "cmlimit": limit,
            "format": "json",
            "cmtype": "page|subcat"
        }
        
        while True:
            try:
                response = self.session.get(self.api_url, params=params)
                data = response.json()
                
                if 'query' in data:
                    members.extend(data['query']['categorymembers'])
                
                if 'continue' not in data:
                    break
                    
                params['cmcontinue'] = data['continue']['cmcontinue']
                time.sleep(0.5)
                
            except Exception as e:
                print(f"Ошибка при получении категории {category}: {e}")
                break
                
        return members
    
    def get_all_car_categories(self):
        """Получает все категории, связанные с автомобилями"""
        categories = [
            "Категория:Автомобили по маркам",
            "Категория:Автомобили по годам",
            "Категория:Легковые автомобили",
            "Категория:Спортивные автомобили",
            "Категория:Внедорожники",
            "Категория:Концепт-кары",
            "Категория:Электромобили",
            "Категория:Автомобили России",
            "Категория:Автомобили СССР",
            "Категория:Автомобили Германии",
            "Категория:Автомобили США",
            "Категория:Автомобили Японии",
            "Категория:Автомобили Италии",
            "Категория:Автомобили Франции",
            "Категория:Автомобили Великобритании",
            "Категория:Автомобили Кореи",
            "Категория:Автомобили Китая",
        ]
        
        all_categories = set(categories)
        processed = set()
        
        # Рекурсивно собираем подкатегории
        while categories:
            category = categories.pop(0)
            if category in processed:
                continue
                
            processed.add(category)
            print(f"Обработка категории: {category}")
            
            members = self.get_category_members(category, limit=500)
            
            for member in members:
                if member['title'].startswith('Категория:'):
                    if member['title'] not in all_categories:
                        all_categories.add(member['title'])
                        categories.append(member['title'])
            
            time.sleep(0.5)
            
            # Ограничиваем глубину поиска
            if len(all_categories) > 200:
                break
        
        return list(all_categories)
    
    def get_article_content(self, title):
        """Получает содержимое статьи"""
        params = {
            "action": "query",
            "titles": title,
            "prop": "extracts|info|categories",
            "explaintext": True,
            "format": "json",
            "inprop": "url"
        }
        
        try:
            response = self.session.get(self.api_url, params=params, timeout=10)
            data = response.json()
            
            pages = data['query']['pages']
            page = list(pages.values())[0]
            
            if 'extract' in page and len(page.get('extract', '')) > 200:
                return {
                    'title': page['title'],
                    'url': page.get('fullurl', ''),
                    'content': page['extract'],
                    'length': len(page['extract'])
                }
        except Exception as e:
            print(f"Ошибка при получении статьи {title}: {e}")
            
        return None
    
    def is_car_article(self, title, content):
        """Проверяет, является ли статья статьей об автомобиле"""
        car_keywords = [
            'автомобиль', 'машина', 'авто', 'двигатель', 'мотор',
            'sedan', 'седан', 'хэтчбек', 'внедорожник', 'кроссовер',
            'купе', 'кабриолет', 'родстер', 'минивэн', 'пикап',
            'произво', 'выпуск', 'модель', 'поколение', 'кузов',
            'колёс', 'привод', 'трансмисс', 'разгон', 'скорость'
        ]
        
        title_lower = title.lower()
        content_lower = content.lower()[:1000]  # Проверяем первые 1000 символов
        
        # Исключаем нежелательные статьи
        exclude_keywords = [
            'категория:', 'список', 'шаблон:', 'файл:', 
            'википедия:', 'проект:', 'обсуждение',
            'компания', 'завод', 'производитель', 'концерн'
        ]
        
        for keyword in exclude_keywords:
            if keyword in title_lower:
                return False
        
        # Проверяем наличие ключевых слов
        keyword_count = sum(1 for keyword in car_keywords if keyword in content_lower)
        
        return keyword_count >= 3
    
    def collect_articles(self, target_count=15000):
        """Собирает статьи об автомобилях"""
        print("Получение категорий...")
        categories = self.get_all_car_categories()
        print(f"Найдено категорий: {len(categories)}")
        
        all_pages = set()
        
        # Собираем все страницы из категорий
        print("\nСбор списка статей...")
        for category in tqdm(categories):
            members = self.get_category_members(category, limit=500)
            for member in members:
                if not member['title'].startswith('Категория:'):
                    all_pages.add(member['title'])
            
            if len(all_pages) >= target_count * 2:  # Берём с запасом
                break
        
        print(f"\nНайдено уникальных страниц: {len(all_pages)}")
        print("Загрузка содержимого статей...")
        
        # Загружаем содержимое статей
        for title in tqdm(list(all_pages)):
            if len(self.articles) >= target_count:
                break
            
            article = self.get_article_content(title)
            
            if article and self.is_car_article(article['title'], article['content']):
                self.articles.append(article)
                
                # Промежуточное сохранение каждые 1000 статей
                if len(self.articles) % 1000 == 0:
                    self.save_articles(f'cars_backup_{len(self.articles)}.json')
                    print(f"\n✓ Собрано статей: {len(self.articles)}")
            
            time.sleep(0.1)  # Чтобы не нагружать сервер
        
        print(f"\n{'='*60}")
        print(f"Сбор завершён! Всего собрано статей: {len(self.articles)}")
        print(f"{'='*60}")
        
        return self.articles
    
    def save_articles(self, filename='wikipedia_cars.json'):
        """Сохраняет статьи в JSON файл"""
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(self.articles, f, ensure_ascii=False, indent=2)
        print(f"Данные сохранены в {filename}")
    
    def save_to_txt(self, filename='wikipedia_cars.txt'):
        """Сохраняет статьи в текстовый файл"""
        with open(filename, 'w', encoding='utf-8') as f:
            for i, article in enumerate(self.articles, 1):
                f.write(f"{'='*80}\n")
                f.write(f"Статья #{i}\n")
                f.write(f"Название: {article['title']}\n")
                f.write(f"URL: {article['url']}\n")
                f.write(f"Длина: {article['length']} символов\n")
                f.write(f"{'='*80}\n\n")
                f.write(article['content'])
                f.write(f"\n\n")
        print(f"Текстовые данные сохранены в {filename}")
    
    def get_statistics(self):
        """Выводит статистику собранных данных"""
        if not self.articles:
            print("Нет собранных статей")
            return
        
        total = len(self.articles)
        total_length = sum(article['length'] for article in self.articles)
        avg_length = total_length / total
        
        print(f"\n{'='*60}")
        print(f"СТАТИСТИКА:")
        print(f"{'='*60}")
        print(f"Всего статей: {total}")
        print(f"Общий объём текста: {total_length:,} символов")
        print(f"Средняя длина статьи: {avg_length:.0f} символов")
        print(f"Самая длинная статья: {max(self.articles, key=lambda x: x['length'])['title']} ({max(article['length'] for article in self.articles):,} символов)")
        print(f"Самая короткая статья: {min(self.articles, key=lambda x: x['length'])['title']} ({min(article['length'] for article in self.articles):,} символов)")
        print(f"{'='*60}\n")


# ИСПОЛЬЗОВАНИЕ
if __name__ == "__main__":
    parser = WikipediaCarParser()
    
    # Собираем 15000 статей
    articles = parser.collect_articles(target_count=15000)
    
    # Сохраняем результаты
    parser.save_articles('wikipedia_cars_15000.json')
    parser.save_to_txt('wikipedia_cars_15000.txt')
    
    # Выводим статистику
    parser.get_statistics()


Получение категорий...
Обработка категории: Категория:Автомобили по маркам
Найдено категорий: 207

Сбор списка статей...


100%|██████████| 207/207 [00:54<00:00,  3.78it/s]



Найдено уникальных страниц: 5025
Загрузка содержимого статей...


 21%|██▏       | 1076/5025 [17:17<1:08:33,  1.04s/it]

Данные сохранены в cars_backup_1000.json

✓ Собрано статей: 1000


...

Ошибка при получении статьи Haval H5: HTTPSConnectionPool(host='ru.wikipedia.org', port=443): Read timed out. (read timeout=10)


100%|█████████▉| 5004/5025 [1:27:56<04:02, 11.55s/it]

Ошибка при получении статьи Vauxhall Meriva: HTTPSConnectionPool(host='ru.wikipedia.org', port=443): Read timed out. (read timeout=10)


100%|██████████| 5025/5025 [1:29:07<00:00,  1.06s/it]


Сбор завершён! Всего собрано статей: 4619
Данные сохранены в wikipedia_cars_15000.json
Текстовые данные сохранены в wikipedia_cars_15000.txt

СТАТИСТИКА:
Всего статей: 4619
Общий объём текста: 19,341,849 символов
Средняя длина статьи: 4187 символов
Самая длинная статья: Mercedes-Benz W213 (110,317 символов)
Самая короткая статья: Ford CX (220 символов)



In [5]:
import requests
from bs4 import BeautifulSoup
import json
import time
import random
from datetime import datetime
from tqdm import tqdm
import re
from urllib.parse import urljoin, urlparse, parse_qs
import csv

class AvitoCarParser:
    def __init__(self):
        self.base_url = "https://www.avito.ru"
        self.session = requests.Session()
        
        # Ротация User-Agent для обхода блокировок
        self.user_agents = [
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:121.0) Gecko/20100101 Firefox/121.0',
        ]
        
        self.cars_data = []
        
    def get_headers(self):
        """Генерирует случайные headers"""
        return {
            'User-Agent': random.choice(self.user_agents),
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
            'Accept-Encoding': 'gzip, deflate, br',
            'DNT': '1',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1',
            'Sec-Fetch-Dest': 'document',
            'Sec-Fetch-Mode': 'navigate',
            'Sec-Fetch-Site': 'none',
            'Cache-Control': 'max-age=0',
        }
    
    def get_regions(self):
        """Список основных регионов для парсинга"""
        return [
            ('moskva', 'Москва'),
            ('sankt-peterburg', 'Санкт-Петербург'),
            ('rossiya', 'Вся Россия'),
            ('novosibirsk', 'Новосибирск'),
            ('ekaterinburg', 'Екатеринбург'),
            ('nizhniy_novgorod', 'Нижний Новгород'),
            ('kazan', 'Казань'),
            ('chelyabinsk', 'Челябинск'),
            ('omsk', 'Омск'),
            ('samara', 'Самара'),
            ('rostov-na-donu', 'Ростов-на-Дону'),
            ('ufa', 'Уфа'),
            ('krasnoyarsk', 'Красноярск'),
            ('voronezh', 'Воронеж'),
            ('perm', 'Пермь'),
            ('volgograd', 'Волгоград'),
        ]
    
    def build_search_url(self, region, page=1, params=None):
        """Строит URL для поиска автомобилей"""
        base_params = {
            'p': page,
            's': 104,  # Сортировка по дате
        }
        
        if params:
            base_params.update(params)
        
        # URL для категории "Автомобили"
        url = f"{self.base_url}/{region}/avtomobili"
        
        # Добавляем параметры
        if base_params:
            param_str = '&'.join([f"{k}={v}" for k, v in base_params.items()])
            url = f"{url}?{param_str}"
        
        return url
    
    def parse_price(self, price_text):
        """Извлекает цену из текста"""
        if not price_text:
            return None
        
        # Убираем все кроме цифр
        price = re.sub(r'[^\d]', '', price_text)
        return int(price) if price else None
    
    def parse_listing_page(self, url, region_name):
        """Парсит страницу с объявлениями"""
        try:
            time.sleep(random.uniform(2, 4))  # Задержка между запросами
            
            response = self.session.get(url, headers=self.get_headers(), timeout=15)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Ищем объявления (структура может меняться)
            items = soup.find_all('div', {'data-marker': 'item'})
            
            if not items:
                # Альтернативный поиск
                items = soup.find_all('div', class_=re.compile('iva-item'))
            
            cars = []
            
            for item in items:
                try:
                    car_data = self.parse_item(item, region_name)
                    if car_data:
                        cars.append(car_data)
                except Exception as e:
                    print(f"Ошибка парсинга объявления: {e}")
                    continue
            
            return cars
            
        except requests.exceptions.RequestException as e:
            print(f"Ошибка запроса {url}: {e}")
            return []
        except Exception as e:
            print(f"Общая ошибка при парсинге страницы: {e}")
            return []
    
    def parse_item(self, item, region_name):
        """Извлекает данные из одного объявления"""
        data = {
            'region': region_name,
            'parse_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
        
        # Заголовок и ссылка
        title_elem = item.find('a', {'data-marker': 'item-title'}) or item.find('a', itemprop='url')
        if title_elem:
            data['title'] = title_elem.get_text(strip=True)
            data['url'] = urljoin(self.base_url, title_elem.get('href', ''))
            
            # Извлекаем ID объявления из URL
            match = re.search(r'_(\d+)$', data['url'])
            if match:
                data['id'] = match.group(1)
        else:
            return None
        
        # Цена
        price_elem = item.find('span', {'data-marker': 'item-price'}) or item.find('meta', itemprop='price')
        if price_elem:
            if price_elem.name == 'meta':
                data['price'] = int(price_elem.get('content', 0))
            else:
                data['price'] = self.parse_price(price_elem.get_text())
        
        # Описание
        description_elem = item.find('div', {'data-marker': 'item-description'}) or item.find('div', class_=re.compile('description'))
        if description_elem:
            data['description'] = description_elem.get_text(strip=True)
        
        # Адрес
        address_elem = item.find('div', {'data-marker': 'item-address'}) or item.find('div', class_=re.compile('geo'))
        if address_elem:
            data['address'] = address_elem.get_text(strip=True)
        
        # Дата публикации
        date_elem = item.find('div', {'data-marker': 'item-date'}) or item.find('div', class_=re.compile('date'))
        if date_elem:
            data['date_published'] = date_elem.get_text(strip=True)
        
        # Параметры (год, пробег, и т.д.)
        params_elem = item.find_all('span', class_=re.compile('param'))
        if params_elem:
            params_text = ' '.join([p.get_text(strip=True) for p in params_elem])
            data['params'] = params_text
            
            # Извлекаем год
            year_match = re.search(r'(\d{4})\s*г', params_text)
            if year_match:
                data['year'] = int(year_match.group(1))
            
            # Извлекаем пробег
            mileage_match = re.search(r'(\d+)\s*км', params_text.replace(' ', ''))
            if mileage_match:
                data['mileage'] = int(mileage_match.group(1))
        
        # Изображение
        img_elem = item.find('img', {'data-marker': 'item-photo'}) or item.find('img', itemprop='image')
        if img_elem:
            data['image_url'] = img_elem.get('src') or img_elem.get('data-src')
        
        return data
    
    def parse_detailed_page(self, url):
        """Парсит детальную страницу объявления"""
        try:
            time.sleep(random.uniform(3, 5))
            
            response = self.session.get(url, headers=self.get_headers(), timeout=15)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.content, 'html.parser')
            
            details = {}
            
            # Полное описание
            description_elem = soup.find('div', {'data-marker': 'item-view/item-description'})
            if description_elem:
                details['full_description'] = description_elem.get_text(strip=True)
            
            # Характеристики автомобиля
            params = {}
            params_list = soup.find_all('li', class_=re.compile('params-paramsList'))
            
            for param in params_list:
                try:
                    key_elem = param.find('span', class_=re.compile('key'))
                    value_elem = param.find('span', class_=re.compile('value'))
                    
                    if key_elem and value_elem:
                        key = key_elem.get_text(strip=True).replace(':', '')
                        value = value_elem.get_text(strip=True)
                        params[key] = value
                except:
                    continue
            
            details['params'] = params
            
            # Извлекаем конкретные характеристики
            if 'Марка' in params:
                details['brand'] = params['Марка']
            if 'Модель' in params:
                details['model'] = params['Модель']
            if 'Год выпуска' in params:
                details['year'] = params['Год выпуска']
            if 'Пробег' in params:
                details['mileage'] = params['Пробег']
            if 'Кузов' in params:
                details['body_type'] = params['Кузов']
            if 'Цвет' in params:
                details['color'] = params['Цвет']
            if 'Двигатель' in params:
                details['engine'] = params['Двигатель']
            if 'Коробка передач' in params:
                details['transmission'] = params['Коробка передач']
            if 'Привод' in params:
                details['drive'] = params['Привод']
            if 'Руль' in params:
                details['steering_wheel'] = params['Руль']
            if 'Состояние' in params:
                details['condition'] = params['Состояние']
            if 'Владельцев по ПТС' in params:
                details['owners'] = params['Владельцев по ПТС']
            
            # Номер телефона (обычно скрыт, требует дополнительный запрос)
            # Контакты продавца
            seller_elem = soup.find('div', {'data-marker': 'seller-info/name'})
            if seller_elem:
                details['seller_name'] = seller_elem.get_text(strip=True)
            
            # Все изображения
            images = []
            img_elements = soup.find_all('div', class_=re.compile('gallery-img'))
            for img in img_elements:
                img_tag = img.find('img')
                if img_tag and img_tag.get('src'):
                    images.append(img_tag.get('src'))
            
            details['images'] = images
            
            # VIN номер (если указан)
            vin_elem = soup.find('span', string=re.compile('VIN'))
            if vin_elem:
                vin_parent = vin_elem.find_parent()
                if vin_parent:
                    vin_text = vin_parent.get_text(strip=True)
                    vin_match = re.search(r'[A-HJ-NPR-Z0-9]{17}', vin_text)
                    if vin_match:
                        details['vin'] = vin_match.group(0)
            
            return details
            
        except Exception as e:
            print(f"Ошибка парсинга детальной страницы {url}: {e}")
            return {}
    
    def collect_cars(self, target_count=15000, max_pages_per_region=50, detailed=False):
        """Собирает объявления об автомобилях"""
        print("🚗 Начинаем сбор объявлений с Avito...")
        print(f"Цель: {target_count} объявлений\n")
        
        regions = self.get_regions()
        total_collected = 0
        
        for region_slug, region_name in regions:
            if total_collected >= target_count:
                break
            
            print(f"\n{'='*60}")
            print(f"Регион: {region_name}")
            print(f"{'='*60}")
            
            for page in range(1, max_pages_per_region + 1):
                if total_collected >= target_count:
                    break
                
                url = self.build_search_url(region_slug, page=page)
                print(f"Страница {page}: {url}")
                
                cars = self.parse_listing_page(url, region_name)
                
                if not cars:
                    print(f"⚠️  Нет объявлений на странице {page}, переходим к следующему региону")
                    break
                
                # Если нужна детальная информация
                if detailed:
                    print(f"Загрузка детальной информации...")
                    for car in tqdm(cars):
                        if 'url' in car:
                            details = self.parse_detailed_page(car['url'])
                            car.update(details)
                            time.sleep(random.uniform(2, 4))
                
                self.cars_data.extend(cars)
                total_collected += len(cars)
                
                print(f"✓ Собрано {len(cars)} объявлений (всего: {total_collected})")
                
                # Промежуточное сохранение каждые 1000 объявлений
                if total_collected % 1000 == 0:
                    self.save_to_json(f'avito_cars_backup_{total_collected}.json')
                    print(f"💾 Создан бэкап: {total_collected} объявлений")
                
                # Увеличенная задержка между страницами
                time.sleep(random.uniform(3, 6))
        
        print(f"\n{'='*60}")
        print(f"✅ Сбор завершён! Всего собрано: {len(self.cars_data)} объявлений")
        print(f"{'='*60}\n")
        
        return self.cars_data
    
    def save_to_json(self, filename='avito_cars.json'):
        """Сохраняет данные в JSON"""
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(self.cars_data, f, ensure_ascii=False, indent=2)
        print(f"💾 Данные сохранены в {filename}")
    
    def save_to_csv(self, filename='avito_cars.csv'):
        """Сохраняет данные в CSV"""
        if not self.cars_data:
            print("Нет данных для сохранения")
            return
        
        # Определяем все возможные поля
        all_fields = set()
        for car in self.cars_data:
            all_fields.update(car.keys())
        
        all_fields = sorted(list(all_fields))
        
        with open(filename, 'w', encoding='utf-8', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=all_fields)
            writer.writeheader()
            
            for car in self.cars_data:
                # Преобразуем списки и словари в строки
                row = {}
                for key, value in car.items():
                    if isinstance(value, (list, dict)):
                        row[key] = json.dumps(value, ensure_ascii=False)
                    else:
                        row[key] = value
                writer.writerow(row)
        
        print(f"💾 Данные сохранены в {filename}")
    
    def save_to_txt(self, filename='avito_cars.txt'):
        """Сохраняет данные в текстовый файл"""
        with open(filename, 'w', encoding='utf-8') as f:
            for i, car in enumerate(self.cars_data, 1):
                f.write(f"{'='*80}\n")
                f.write(f"Объявление #{i}\n")
                f.write(f"{'='*80}\n\n")
                
                for key, value in car.items():
                    if isinstance(value, (list, dict)):
                        f.write(f"{key}:\n{json.dumps(value, ensure_ascii=False, indent=2)}\n\n")
                    else:
                        f.write(f"{key}: {value}\n")
                
                f.write(f"\n\n")
        
        print(f"💾 Данные сохранены в {filename}")
    
    def get_statistics(self):
        """Выводит статистику собранных данных"""
        if not self.cars_data:
            print("Нет данных для статистики")
            return
        
        total = len(self.cars_data)
        
        # Статистика по ценам
        prices = [car.get('price') for car in self.cars_data if car.get('price')]
        avg_price = sum(prices) / len(prices) if prices else 0
        min_price = min(prices) if prices else 0
        max_price = max(prices) if prices else 0
        
        # Статистика по годам
        years = [car.get('year') for car in self.cars_data if car.get('year')]
        
        # Статистика по регионам
        regions = {}
        for car in self.cars_data:
            region = car.get('region', 'Неизвестно')
            regions[region] = regions.get(region, 0) + 1
        
        print(f"\n{'='*60}")
        print(f"📊 СТАТИСТИКА СОБРАННЫХ ДАННЫХ")
        print(f"{'='*60}")
        print(f"Всего объявлений: {total}")
        print(f"\n💰 Цены:")
        print(f"  Средняя цена: {avg_price:,.0f} ₽")
        print(f"  Минимальная: {min_price:,.0f} ₽")
        print(f"  Максимальная: {max_price:,.0f} ₽")
        
        if years:
            print(f"\n📅 Годы выпуска:")
            print(f"  Средний год: {sum(years)/len(years):.0f}")
            print(f"  Самый старый: {min(years)}")
            print(f"  Самый новый: {max(years)}")
        
        print(f"\n🌍 По регионам:")
        for region, count in sorted(regions.items(), key=lambda x: x[1], reverse=True)[:10]:
            print(f"  {region}: {count}")
        
        print(f"{'='*60}\n")
    
    def filter_by_price(self, min_price=None, max_price=None):
        """Фильтрует автомобили по цене"""
        filtered = []
        for car in self.cars_data:
            price = car.get('price')
            if price:
                if min_price and price < min_price:
                    continue
                if max_price and price > max_price:
                    continue
                filtered.append(car)
        return filtered
    
    def filter_by_year(self, min_year=None, max_year=None):
        """Фильтрует автомобили по году"""
        filtered = []
        for car in self.cars_data:
            year = car.get('year')
            if year:
                if min_year and year < min_year:
                    continue
                if max_year and year > max_year:
                    continue
                filtered.append(car)
        return filtered
    
    def search_by_brand(self, brand):
        """Ищет автомобили по марке"""
        brand_lower = brand.lower()
        results = []
        
        for car in self.cars_data:
            title = car.get('title', '').lower()
            car_brand = car.get('brand', '').lower()
            
            if brand_lower in title or brand_lower in car_brand:
                results.append(car)
        
        return results


# ============================================================================
# ИСПОЛЬЗОВАНИЕ
# ============================================================================

if __name__ == "__main__":
    parser = AvitoCarParser()
    
    print("🚗 ПАРСЕР AVITO - АВТОМОБИЛИ")
    print("="*60)
    print("\nВыберите режим работы:")
    print("1. Быстрый сбор (только основная информация) - РЕКОМЕНДУЕТСЯ")
    print("2. Детальный сбор (с переходом на каждую страницу) - МЕДЛЕННО")
    
    mode = input("\nВведите номер режима (1 или 2): ").strip()
    
    detailed = (mode == "2")
    
    # Собираем данные
    cars = parser.collect_cars(
        target_count=15000, 
        max_pages_per_region=100,  # Максимум страниц на регион
        detailed=detailed
    )
    
    # Сохраняем результаты
    parser.save_to_json('avito_cars_15000.json')
    parser.save_to_csv('avito_cars_15000.csv')
    parser.save_to_txt('avito_cars_15000.txt')
    
    # Выводим статистику
    parser.get_statistics()
    
    # Примеры использования фильтров
    print("\n📌 ПРИМЕРЫ ФИЛЬТРАЦИИ:\n")
    
    # Автомобили от 500k до 1M
    expensive = parser.filter_by_price(min_price=500000, max_price=1000000)
    print(f"Автомобили от 500k до 1M: {len(expensive)} шт.")
    
    # Автомобили 2020 года и новее
    new_cars = parser.filter_by_year(min_year=2020)
    print(f"Автомобили 2020+ года: {len(new_cars)} шт.")
    
    # Поиск по марке
    toyota = parser.search_by_brand("Toyota")
    print(f"Toyota: {len(toyota)} шт.")



🚗 ПАРСЕР AVITO - АВТОМОБИЛИ

Выберите режим работы:
1. Быстрый сбор (только основная информация) - РЕКОМЕНДУЕТСЯ
2. Детальный сбор (с переходом на каждую страницу) - МЕДЛЕННО
🚗 Начинаем сбор объявлений с Avito...
Цель: 15000 объявлений


Регион: Москва
Страница 1: https://www.avito.ru/moskva/avtomobili?p=1&s=104
✓ Собрано 50 объявлений (всего: 50)
Страница 2: https://www.avito.ru/moskva/avtomobili?p=2&s=104
✓ Собрано 50 объявлений (всего: 100)
Страница 3: https://www.avito.ru/moskva/avtomobili?p=3&s=104
✓ Собрано 50 объявлений (всего: 150)
Страница 4: https://www.avito.ru/moskva/avtomobili?p=4&s=104
✓ Собрано 50 объявлений (всего: 200)
Страница 5: https://www.avito.ru/moskva/avtomobili?p=5&s=104
✓ Собрано 50 объявлений (всего: 250)
Страница 6: https://www.avito.ru/moskva/avtomobili?p=6&s=104
✓ Собрано 36 объявлений (всего: 286)
Страница 7: https://www.avito.ru/moskva/avtomobili?p=7&s=104
✓ Собрано 33 объявлений (всего: 319)
Страница 8: https://www.avito.ru/moskva/avtomobili?p=8&s=104
✓

In [9]:
# python/01_corpus/corpus_builder.py

import json
import os
from pathlib import Path
from typing import List, Dict
import hashlib

class CorpusBuilder:
    """
    Объединяет данные из разных источников в единый корпус
    """
    
    def __init__(self, output_dir='data/processed'):
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(parents=True, exist_ok=True)
        self.documents = []
        self.doc_id_counter = 0
        
    def load_wikipedia_articles(self, wiki_file):
        """Загружает статьи Wikipedia"""
        with open(wiki_file, 'r', encoding='utf-8') as f:
            articles = json.load(f)
        
        for article in articles:
            self.documents.append({
                'doc_id': self.doc_id_counter,
                'source': 'wikipedia',
                'title': article['title'],
                'url': article['url'],
                'text': article['content'],
                'length': article['length']
            })
            self.doc_id_counter += 1
    
    def load_avito_ads(self, avito_file):
        """Загружает объявления Avito"""
        with open(avito_file, 'r', encoding='utf-8') as f:
            ads = json.load(f)
        
        for ad in ads:
            # Объединяем все текстовые поля
            text = f"{ad.get('title', '')} {ad.get('description', '')} {ad.get('params', '')}"
            
            self.documents.append({
                'doc_id': self.doc_id_counter,
                'source': 'avito',
                'title': ad.get('title', 'Без названия'),
                'url': ad.get('url', ''),
                'text': text,
                'price': ad.get('price'),
                'region': ad.get('region'),
                'year': ad.get('year')
            })
            self.doc_id_counter += 1
    
    def save_corpus(self):
        """Сохраняет единый корпус"""
        # JSON формат (для Python)
        output_json = self.output_dir / 'corpus.json'
        with open(output_json, 'w', encoding='utf-8') as f:
            json.dump(self.documents, f, ensure_ascii=False, indent=2)
        
        # Текстовый формат (для C++)
        output_txt = self.output_dir / 'corpus.txt'
        with open(output_txt, 'w', encoding='utf-8') as f:
            for doc in self.documents:
                # Формат: doc_id|source|title|text
                f.write(f"{doc['doc_id']}|{doc['source']}|{doc['title']}|{doc['text']}\n")
        
        # Метаданные
        output_meta = self.output_dir / 'corpus_meta.json'
        with open(output_meta, 'w', encoding='utf-8') as f:
            json.dump({
                'total_documents': len(self.documents),
                'sources': {
                    'wikipedia': sum(1 for d in self.documents if d['source'] == 'wikipedia'),
                    'avito': sum(1 for d in self.documents if d['source'] == 'avito')
                },
                'total_size': sum(len(d['text']) for d in self.documents)
            }, f, ensure_ascii=False, indent=2)
        
        print(f"✅ Корпус сохранен: {len(self.documents)} документов")
        print(f"   JSON: {output_json}")
        print(f"   TXT: {output_txt}")
        print(f"   META: {output_meta}")
    
    def get_statistics(self):
        """Статистика корпуса"""
        total = len(self.documents)
        total_chars = sum(len(d['text']) for d in self.documents)
        avg_length = total_chars / total if total > 0 else 0
        
        print(f"\n📊 СТАТИСТИКА КОРПУСА:")
        print(f"{'='*60}")
        print(f"Всего документов: {total}")
        print(f"Общий размер: {total_chars:,} символов")
        print(f"Средняя длина: {avg_length:.0f} символов")
        print(f"Wikipedia: {sum(1 for d in self.documents if d['source'] == 'wikipedia')}")
        print(f"Avito: {sum(1 for d in self.documents if d['source'] == 'avito')}")
        print(f"{'='*60}\n")

# Использование
if __name__ == "__main__":
    builder = CorpusBuilder()
    builder.load_wikipedia_articles('data/raw/wikipedia_cars_15000.json')
    builder.load_avito_ads('data/raw/avito_cars_15000.json')
    builder.get_statistics()
    builder.save_corpus()



📊 СТАТИСТИКА КОРПУСА:
Всего документов: 19642
Общий размер: 19,911,281 символов
Средняя длина: 1014 символов
Wikipedia: 4619
Avito: 15023

✅ Корпус сохранен: 19642 документов
   JSON: data/processed/corpus.json
   TXT: data/processed/corpus.txt
   META: data/processed/corpus_meta.json


In [5]:
import json
import os

def clean_text(text):
    """Очищает текст от служебных символов"""
    if not text:
        return ""
    # Заменяем | на пробел (это разделитель в нашем формате)
    text = text.replace('|', ' ')
    # Убираем множественные пробелы
    text = ' '.join(text.split())
    return text

def process_json_file(json_path, start_id=0):
    """Обрабатывает один JSON файл"""
    print(f"📂 Обработка: {json_path}")
    
    if not os.path.exists(json_path):
        print(f"❌ Файл не найден: {json_path}")
        return [], start_id
    
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except json.JSONDecodeError as e:
        print(f"❌ Ошибка JSON: {e}")
        return [], start_id
    
    # Определяем тип источника по структуре первого элемента
    if data and isinstance(data, list):
        first_item = data[0]
        if 'content' in first_item:
            source_type = 'wikipedia'
        elif 'price' in first_item or 'region' in first_item:
            source_type = 'avito'
        else:
            source_type = 'unknown'
    else:
        source_type = 'unknown'
    
    print(f"  Тип источника: {source_type}")
    
    documents = []
    skipped = 0
    
    for item in data:
        # Определяем источник
        source = item.get('source', source_type)
        
        # Заголовок
        title = item.get('title', '')
        if not title:
            skipped += 1
            continue
        
        # URL
        url = item.get('url', '')
        
        # Текст - разные поля для разных источников
        text = ''
        
        if source == 'wikipedia' or source_type == 'wikipedia':
            # Wikipedia: поле content
            text = item.get('content', item.get('text', ''))
        
        elif source == 'avito' or source_type == 'avito':
            # Avito: собираем из разных полей
            parts = []
            
            # Заголовок (уже есть, но добавим в текст)
            if 'title' in item:
                parts.append(str(item['title']))
            
            # Описание
            if 'description' in item:
                parts.append(str(item['description']))
            
            # Параметры
            if 'params' in item:
                parts.append(str(item['params']))
            
            # Адрес
            if 'address' in item:
                parts.append(f"Адрес: {item['address']}")
            
            # Цена
            if 'price' in item and item['price']:
                parts.append(f"Цена: {item['price']} рублей")
            
            # Регион
            if 'region' in item:
                parts.append(f"Регион: {item['region']}")
            
            # Дата
            if 'date_published' in item:
                parts.append(f"Опубликовано: {item['date_published']}")
            
            # Все дополнительные текстовые поля
            for key in ['year', 'mileage', 'body_type', 'color', 
                       'engine', 'transmission', 'drive', 'condition']:
                if key in item and item[key]:
                    parts.append(f"{key}: {item[key]}")
            
            text = ' '.join(str(p) for p in parts if p)
        
        else:
            # Универсальный вариант
            text = item.get('content', item.get('text', item.get('description', '')))
        
        # Проверяем минимальную длину
        if not text or len(text.strip()) < 20:  # Снижаем порог для Avito
            skipped += 1
            continue
        
        # Очищаем
        title = clean_text(title)
        text = clean_text(text)
        url = clean_text(url)
        source = clean_text(source)
        
        # Добавляем документ
        documents.append({
            'doc_id': start_id,
            'source': source,
            'title': title,
            'url': url,
            'text': text
        })
        
        start_id += 1
    
    print(f"  ✅ Обработано: {len(documents)} документов")
    if skipped > 0:
        print(f"  ⚠️  Пропущено: {skipped} документов (короткие или без заголовка)")
    
    return documents, start_id

def write_corpus(documents, output_path):
    """Записывает документы в формате для C++"""
    print(f"\n💾 Сохранение в: {output_path}")
    
    with open(output_path, 'w', encoding='utf-8') as f:
        for doc in documents:
            # Формат: doc_id|source|title|url|text
            line = f"{doc['doc_id']}|{doc['source']}|{doc['title']}|{doc['url']}|{doc['text']}\n"
            f.write(line)
    
    print(f"  ✅ Сохранено: {len(documents)} документов")

def print_sample_documents(documents, count=3):
    """Выводит примеры документов"""
    print(f"\n📄 ПРИМЕРЫ ДОКУМЕНТОВ (первые {count}):")
    print("=" * 80)
    
    for i, doc in enumerate(documents[:count], 1):
        print(f"\n{i}. [{doc['source']}] {doc['title']}")
        print(f"   URL: {doc['url'][:60]}...")
        print(f"   Текст (первые 200 символов):")
        print(f"   {doc['text'][:200]}...")
    
    print("\n" + "=" * 80)

def main():
    print("=" * 60)
    print("🔄 ПОДГОТОВКА КОРПУСА ДАННЫХ")
    print("=" * 60)
    
    all_documents = []
    next_id = 0
    
    # Обработка Wikipedia
    wiki_path = 'data/raw/wikipedia_cars.json'
    wiki_docs, next_id = process_json_file(wiki_path, next_id)
    all_documents.extend(wiki_docs)
    
    # Обработка Avito
    avito_path = 'data/raw/avito_cars.json'
    avito_docs, next_id = process_json_file(avito_path, next_id)
    all_documents.extend(avito_docs)
    
    print("\n" + "=" * 60)
    print(f"📊 ИТОГО:")
    print(f"  Wikipedia: {len(wiki_docs)} документов")
    print(f"  Avito: {len(avito_docs)} документов")
    print(f"  Всего: {len(all_documents)} документов")
    print("=" * 60)
    
    if len(all_documents) == 0:
        print("\n❌ Нет документов для обработки!")
        print("Проверьте наличие файлов:")
        print("  - data/raw/wikipedia_cars.json")
        print("  - data/raw/avito_cars.json")
        return
    
    # Показываем примеры
    if wiki_docs:
        print("\n📖 Пример Wikipedia:")
        doc = wiki_docs[0]
        print(f"  Заголовок: {doc['title']}")
        print(f"  Текст: {doc['text'][:150]}...")
    
    if avito_docs:
        print("\n🛒 Пример Avito:")
        doc = avito_docs[0]
        print(f"  Заголовок: {doc['title']}")
        print(f"  Текст: {doc['text'][:150]}...")
    
    # Сохраняем
    output_path = 'data/processed/corpus.txt'
    os.makedirs('data/processed', exist_ok=True)
    write_corpus(all_documents, output_path)
    
    # Проверка
    print("\n🔍 ПРОВЕРКА ФАЙЛА:")
    with open(output_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    print(f"  Строк в файле: {len(lines)}")
    
    if lines:
        print(f"\n  Первая строка (Wikipedia):")
        wiki_line = next((l for l in lines if '|wikipedia|' in l), None)
        if wiki_line:
            print(f"    {wiki_line[:200]}...")
        
        print(f"\n  Первая строка (Avito):")
        avito_line = next((l for l in lines if '|avito|' in l), None)
        if avito_line:
            print(f"    {avito_line[:200]}...")
    
    print("\n" + "=" * 60)
    print("✅ Готово! Теперь запустите:")
    print("  ./rebuild_all.sh")
    print("или")
    print("  ./build/tokenizer data/processed/corpus.txt data/processed/tokens.txt")
    print("=" * 60)

if __name__ == '__main__':
    main()

🔄 ПОДГОТОВКА КОРПУСА ДАННЫХ
📂 Обработка: data/raw/wikipedia_cars.json
  Тип источника: wikipedia
  ✅ Обработано: 4619 документов
📂 Обработка: data/raw/avito_cars.json
  Тип источника: avito
  ✅ Обработано: 15023 документов

📊 ИТОГО:
  Wikipedia: 4619 документов
  Avito: 15023 документов
  Всего: 19642 документов

📖 Пример Wikipedia:
  Заголовок: Jeep Grand Commander
  Текст: Jeep Grand Commander — среднеразмерный кроссовер, выпускающийся с 2018 по 2022 год «внедорожным» подразделением Chrysler — Jeep. == История == Автомоб...

🛒 Пример Avito:
  Заголовок: ВАЗ (LADA) Granta 1.6 MT, 2024, 51 218 км
  Текст: ВАЗ (LADA) Granta 1.6 MT, 2024, 51 218 км Адрес: Москва, Покровское Цена: 709650 рублей Регион: Москва Опубликовано: 1 час назад...

💾 Сохранение в: data/processed/corpus.txt
  ✅ Сохранено: 19642 документов

🔍 ПРОВЕРКА ФАЙЛА:
  Строк в файле: 19642

  Первая строка (Wikipedia):
    0|wikipedia|Jeep Grand Commander|https://ru.wikipedia.org/wiki/Jeep_Grand_Commander|Jeep Grand Commander